# Create predcitors of Prophet and DeepAR+ using Target, Related and Item Meta
* **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np


In [13]:
%store -r

In [14]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Hyperparameters

In [15]:
forecastHorizon = 13 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 13 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "W"

A detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)

#### Predictor algorithms

In [16]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [17]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + item_meta_suffix + suffix

In [18]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(PredictorName=prophet_predictorName, 
                                                  AlgorithmArn=prophet_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )




### DeepAR+

In [19]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + item_meta_suffix + suffix

In [20]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(PredictorName=deeparp_predictorName, 
                                                  AlgorithmArn=deepAR_Plus_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )





## Check if the predictors are active


In [21]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [22]:

item_meta_prophet_predictor_arn = prophet_create_predictor_response["PredictorArn"]
item_meta_deepar_predictor_arn = deeparp_create_predictor_response["PredictorArn"]

In [23]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_prophet_predictor_arn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_deepar_predictor_arn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Predictor evaluation

#### Prophet

In [24]:
# Prophet Metrics
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=item_meta_prophet_predictor_arn)
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 101760.49632091868,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.03620056372058531,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.0682126846492059,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.03182387019514716,
                                                                                          'Quantile': 0.1}]}},
                                            

#### DeepAR+

In [27]:
# DeepAR+ Metrics
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=item_meta_deepar_predictor_arn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 100212.15272551766,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.03107221857591701,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06866436074709086,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.03664060091157902,
                                                                                          'Quantile': 0.1}]}},
                                      

In [28]:
%store item_meta_prophet_predictor_arn
%store item_meta_deepar_predictor_arn

Stored 'item_meta_prophet_predictor_arn' (str)
Stored 'item_meta_deepar_predictor_arn' (str)
